Artificial Neural Network

Installing Tensorflow<br>
pip install tensorflow

Installing Keras<br>
pip install --upgrade keras

Part 1 - Data Preprocessing

Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
class_names = ['Exited', 'Not']

Encoding categorical data

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_2 = LabelEncoder()

X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')

X = np.array(columnTransformer.fit_transform(X), dtype = np.str)

X = X[:, 1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Part 2 - Now let's make the ANN!

Importing the Keras libraries and packages

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Initialising the ANN

In [8]:
classifier = Sequential()

Adding the input layer and the first hidden layer

In [9]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

Adding the second hidden layer

In [10]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

Adding the output layer

In [11]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

Compiling the ANN

In [12]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Fitting the ANN to the Training set

In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 4s 4ms/step - loss: 0.4808 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4284 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4223 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4188 - accuracy: 0.8207
Epoch 5/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4164 - accuracy: 0.8276
Epoch 6/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4144 - accuracy: 0.8279
Epoch 7/100
800/800 [==============================] - 4s 5ms/step - loss: 0.4124 - accuracy: 0.8319
Epoch 8/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4118 - accuracy: 0.8329
Epoch 9/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4104 - accuracy: 0.8334
Epoch 10/100
800/800 [==============================] - 2s 3ms/step - loss: 0.4091 - accura

Part 3 - Making predictions and evaluating the model

Predicting the Test set results

In [14]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


Predicting a single new observation


Predict if the customer with the following informations will leave the bank:<br>
Geography: France<br>
Credit Score: 600<br>
Gender: Male<br>
Age: 40<br>
Tenure: 3<br>
Balance: 60000<br>
Number of Products: 2<br>
Has Credit Card: Yes<br>
Is Active Member: Yes<br>
timated Salary: 50000


In [16]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]])

Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1538,   57],
       [ 265,  140]])

In [18]:
import tensorflow as tf
import random
import json
import matplotlib.pyplot as plt
import os
import tempfile


# Let's obtain a temporary storage directory
MODEL_DIR = tempfile.gettempdir()

In [19]:
MODEL_DIR

'/tmp'

In [20]:
# Let's specify the model version
version = 11 

In [21]:
# Let's join the temp model directory with our chosen version number 
# The expected result will be = '\tmp\version number'
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

export_path = /tmp/11



In [22]:
tf.keras.models.save_model(
    classifier,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

INFO:tensorflow:Assets written to: /tmp/11/assets


In [23]:
!ls -l {export_path}

total 104
drwxr-xr-x 2 root root  4096 Apr 13 01:00 assets
-rw-r--r-- 1 root root  8601 Apr 13 01:00 keras_metadata.pb
-rw-r--r-- 1 root root 85001 Apr 13 01:00 saved_model.pb
drwxr-xr-x 2 root root  4096 Apr 13 01:00 variables


In [24]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 11)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          dense_input: Tens

# Serve the model using tensorflow serving

# Let's add tensorflow-model-server package to our list of packages

In [25]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/

# Let's install tensorflow model server:

In [26]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [27]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8502 \
  --model_name=churn_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


# Let's start making requests in TensorFlow Serving

In [28]:
!pip install -q requests

In [29]:
import requests

In [30]:
# Let's create a JSON object and make 3 inference requests
data = json.dumps({"signature_name": "serving_default", "instances": X_test[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... 87704583, 0.9687383975403476, -0.4247867388302741]]}


Run it on your local machine

In [31]:
''' headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8502/v11/models/churn_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(0,3):
  show(i, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[y_pred[i]], y_pred[i])) '''

' headers = {"content-type": "application/json"}\njson_response = requests.post(\'http://localhost:8502/v11/models/churn_model:predict\', data=data, headers=headers)\npredictions = json.loads(json_response.text)[\'predictions\']\n\nfor i in range(0,3):\n  show(i, \'The model thought this was a {} (class {}), and it was actually a {} (class {})\'.format(\n    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[y_pred[i]], y_pred[i])) '